# BeautifulSoup 爬虫知识

   * requests 用来获取页面内容
   * BeautifulSoup

* 安装           
pip install requests         
pip install bs4

## 一、Beautiful Soup的四大对象种类

Beautiful Soup将复杂HTML文档转换成一个复杂的树形结构,每个节点都是Python对象,所有对象可以归纳为4种:    
* Tag    
* NavigableString   
* BeautifulSoup   
* Comment

In [1]:
from bs4 import BeautifulSoup

html = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title" name="dromouse"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1"><!-- Elsie --></a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>
<p class="story">...</p>
"""

soup = BeautifulSoup(html)

In [3]:
# soup.prettify() 函数：格式化输出soup对象内容
print(soup.prettify())

<html>
 <head>
  <title>
   The Dormouse's story
  </title>
 </head>
 <body>
  <p class="title" name="dromouse">
   <b>
    The Dormouse's story
   </b>
  </p>
  <p class="story">
   Once upon a time there were three little sisters; and their names were
   <a class="sister" href="http://example.com/elsie" id="link1">
    <!-- Elsie -->
   </a>
   ,
   <a class="sister" href="http://example.com/lacie" id="link2">
    Lacie
   </a>
   and
   <a class="sister" href="http://example.com/tillie" id="link3">
    Tillie
   </a>
   ;
and they lived at the bottom of a well.
  </p>
  <p class="story">
   ...
  </p>
 </body>
</html>


### (1) Tag   

Tag就是HTML中的一个个标签。例如上面定义的html里的head，title，a等

In [4]:
print(soup.title)

<title>The Dormouse's story</title>


In [5]:
print(soup.head)

<head><title>The Dormouse's story</title></head>


对于Tag，她有两个重要的属性，分别是**name和attrs**

In [6]:
# name

print(soup.name)           #soup对象本身也有name，为[document]
print(soup.head.name)  #其他内部标签，name即为标签本身的名称

[document]
head


In [7]:
# attrs

print(soup.p.attrs)

{'class': ['title'], 'name': 'dromouse'}


这里原html中有 

    <p class="title" name="dromouse"><b>The Dormouse's story</b></p>
           

我们把p标签的所有属性打印输出了出来，得到一个字典  


In [8]:
#如果我们想要获取单独某个属性，就可以采用字典的方式去访问
print(soup.p['name'])

dromouse


In [9]:
#也可以使用get方法
print(soup.p.get('name'))

dromouse


### (2) NavigableString    

In [10]:
#既然已经得到了标签，那么获取标签里的文字也很简单，用 .string 即可
soup.p.string

"The Dormouse's story"

In [11]:
#NavigableString其实就是可以遍历的字符串
type(soup.p.string)

bs4.element.NavigableString

### (3) BeautifulSoup

BeautifulSoup 对象表示的是一个文档的全部内容。    
大部分时候,可以把它当作 Tag 对象，是一个特殊的 Tag，我们可以分别获取它的类型，名称，以及属性。

In [12]:
type(soup.name)

str

In [13]:
soup.name

'[document]'

In [14]:
soup.attrs

{}

### (4) Comment

Comment 对象是一个特殊类型的 NavigableString 对象，其实输出的内容仍然不包括注释符号，但是如果不好好处理它，可能会对我们的文本处理造成意想不到的麻烦。

In [17]:
print(soup.a)
print(soup.a.string)
print(type(soup.a.string))

<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>
 Elsie 
<class 'bs4.element.Comment'>


a标签里的内容实际上是注释，但是我们利用 .string 来输出它的内容，发现注释符号去掉了，所以会带来不必要的麻烦

输出的类型是一个 Comment 类型，所以，我们在使用前最好做一下判断：

In [26]:
import bs4

#注意：此处若不引用bs4库，会报错        name 'bs4' is not defined
#因为 from bs4 import BeautifulSoup          import的只是bs4库的一小部分，
#而if语句用到的是没有导入的一部分，就再加上bs4库的引用，再运行就没有再报错了。

if (type(soup.a.string)!=bs4.element.Comment):
    print(soup.a.string)
else:
    print('Comment(',soup.a.string,')')

Comment(  Elsie  )


## 二、遍历文档树

### (1)直接子节点

* **.contents**    

In [28]:
# tag的 .contents 属性可以将tag的子节点以列表的方式输出
soup.head.contents

[<title>The Dormouse's story</title>]

In [34]:
# 输出方式为列表，可以使用列表索引来获取它的某个元素
soup.head.contents[0]

<p class="title" name="dromouse"><b>The Dormouse's story</b></p>

* **.children**    
它返回的不是一个list，而是迭代器，我们可以通过遍历获取所有子节点

In [35]:
# .children是一个 list 生成器对象
soup.head.children

In [37]:
# 遍历一下
for child in soup.body.children:
    print(child)



<p class="title" name="dromouse"><b>The Dormouse's story</b></p>


<p class="story">Once upon a time there were three little sisters; and their names were
<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>,
<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>


<p class="story">...</p>




### (2) 所有子孙节点

* **.descendants**    

.contents 和 .children 属性仅包含tag的直接子节点， .descendants属性可以对所有tag的子孙节点进行循环递归，和 children 类似，我们也需要遍历获取其内容

In [ ]:
for child in soup.descendants:
    print(child)

运行发现所有节点都被打印出来了，先是最外层的HTML标签，其次从head标签一个个剥离，以此类推

### (3) 节点内容

* **.string**   

如果tag只有一个 NavigableString 类型子节点,那么这个tag可以使用 .string 得到子节点。如果一个tag仅有一个子节点,那么这个tag也可以使用 .string 方法,输出结果与当前唯一子节点的 .string 结果相同。

即：如果一个标签里面没有标签了，那么 .string 就会返回标签里面的内容。如果标签里面只有唯一的一个标签了，那么 .string 也会返回最里面的内容。例如

In [39]:
print(soup.head.string)
print(soup.title.string)

The Dormouse's story
The Dormouse's story


如果tag包含了多个子节点，tag就无法确定string方法应该调用哪个子节点的内容，**.string的输出结果是None**

In [41]:
print(soup.html.string)

None


### (4) 多个内容

* **.strings**    
获取多个内容，不过需要遍历获取

In [45]:
for string in soup.strings:
    print(repr(string))    #repr函数，返回对象的string格式(包括换行符等)

"The Dormouse's story"
'\n'
'\n'
"The Dormouse's story"
'\n'
'Once upon a time there were three little sisters; and their names were\n'
',\n'
'Lacie'
' and\n'
'Tillie'
';\nand they lived at the bottom of a well.'
'\n'
'...'
'\n'


* **.stripped_strings**    
可以去除多余空白内容

In [46]:
for string in soup.stripped_strings:
    print(repr(string))

"The Dormouse's story"
"The Dormouse's story"
'Once upon a time there were three little sisters; and their names were'
','
'Lacie'
'and'
'Tillie'
';\nand they lived at the bottom of a well.'
'...'


### (5) 父节点

* **.parent**

In [47]:
p = soup.p
p.parent.name

'body'

In [48]:
content = soup.head.title.string
content.parent.name

'title'

### (6) 全部父节点
* **.parents**

In [49]:
content = soup.head.title.string
for parent in content.parents:
    print(parent.name)

title
head
html
[document]


### (7) 兄弟节点

* **.next_sibling 和 .previous_sibling**    

注意：实际文档中的tag的 .next_sibling 和 .previous_sibling 属性通常是字符串或空白，因为空白或者换行也可以被视作一个节点，所以得到的结果可能是空白或者换行

In [50]:
print(repr(soup.p.next_sibling))

'\n'


In [54]:
print(soup.p.prev_sibling)

None


In [55]:
soup.p.next_sibling.next_sibling

<p class="story">Once upon a time there were three little sisters; and their names were
<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>,
<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

### (8) 全部兄弟节点   
* **.next_siblings 和 .previous_siblings**

### (9) 前后节点

* **.next_element 和 .previous_element**    
与 .next_sibling 和 .previous_sibling 相区别，它并不是针对兄弟节点，而是在所有节点，不分层次

In [56]:
soup.head.next_element

<title>The Dormouse's story</title>

### (10) 前后所有节点

* **.next_elements 和 .previous_elements**

## 三、搜索文档树

### (1) find_all(name, attrs, recursive, text, **kwargs)

find_all 方法搜索当前tag的所有tag子节点，并判断是否符合过滤器的条件

#### 1.name参数：查找所有名字为name的tag

* **传字符串**：最简单的过滤器是字符串，在搜索方法中传入一个字符串参数，Beautiful Soup会查找与字符串完整匹配的内容

In [57]:
soup.find_all('b')

[<b>The Dormouse's story</b>]

In [59]:
soup.find_all('p')

[<p class="title" name="dromouse"><b>The Dormouse's story</b></p>,
 <p class="story">Once upon a time there were three little sisters; and their names were
 <a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
 and they lived at the bottom of a well.</p>,
 <p class="story">...</p>]

* **传正则表达式**：如果传入正则表达式，Beautiful Soup会通过正则表达式的 **match()** 来匹配内容

In [60]:
import re
for tag in soup.find_all(re.compile("^b")):
    print(tag.name)

body
b


* **传列表**：如果传入列表参数,Beautiful Soup会将与**列表中任一元素匹配**的内容返回

In [61]:
soup.find_all(["a", "b"])

[<b>The Dormouse's story</b>,
 <a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

* **传True**：True 可以匹配任何值,下面代码查找到所有的tag,但是不会返回字符串节点

In [62]:
for tag in soup.find_all(True):
    print(tag.name)

html
head
title
body
p
b
p
a
a
a
p


* **传方法**：如果没有合适过滤器,那么还可以定义一个方法,方法只接受一个元素参数,如果这个方法返回 True 表示当前元素匹配并且被找到,如果不是则反回 False

In [65]:
def has_class_but_no_id(tag):
    return tag.has_attr('class') and (not tag.has_attr('id'))

In [66]:
soup.find_all(has_class_but_no_id)

[<p class="title" name="dromouse"><b>The Dormouse's story</b></p>,
 <p class="story">Once upon a time there were three little sisters; and their names were
 <a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
 and they lived at the bottom of a well.</p>,
 <p class="story">...</p>]

#### 2.keyword参数

In [67]:
soup.find_all(id='link2')

[<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>]

In [68]:
soup.find_all(href=re.compile("elsie"))

[<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>]

In [69]:
soup.find_all(href=re.compile("elsie"), id='link1')

[<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>]

In [70]:
# 使用class过滤，可以加下划线！
soup.find_all("a", class_="sister")

[<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

In [71]:
# 有些tag属性在搜索不能使用,比如HTML5中的 data-* 属性
data_soup = BeautifulSoup('<div data-foo="value">foo!</div>')
data_soup.find_all(data-foo="value")

SyntaxError: keyword can't be an expression (<ipython-input-71-a66666d9b6f1>, line 3)

In [73]:
#但是可以通过 find_all() 方法的 attrs 参数定义一个字典参数来搜索包含特殊属性的tag
data_soup = BeautifulSoup('<div data-foo="value">foo!</div>')
data_soup.find_all(attrs={"data-foo": "value"})

[<div data-foo="value">foo!</div>]

#### 3.text参数

通过 text 参数可以搜搜文档中的字符串内容。与 name 参数的可选值一样, text 参数接受 字符串 , 正则表达式 , 列表, True

In [83]:
soup.find_all(text=" Elsie ")

[' Elsie ']

In [84]:
soup.find_all(text=["Tillie", "Elsie", "Lacie"])

['Lacie', 'Tillie']

In [85]:
soup.find_all(text=re.compile("Dormouse"))

["The Dormouse's story", "The Dormouse's story"]

#### 4.limit参数

find_all() 方法返回全部的搜索结构,如果文档树很大那么搜索会很慢.如果我们不需要全部结果,可以使用 limit 参数限制返回结果的数量.

In [86]:
soup.find_all("a", limit=2)

[<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>]

#### 5.recursive 参数

调用tag的 find_all() 方法时,Beautiful Soup会检索当前tag的所有子孙节点,如果只想搜索tag的直接子节点,可以使用参数 recursive=False 

### (2) find(name , attrs , recursive , text , **kwargs)    

它与 find_all() 方法唯一的区别是 find_all() 方法的返回结果是值包含一个元素的列表,而 find() 方法直接返回结果

### (3) find_parents()  find_parent()    

find_all() 和 find() 只搜索当前节点的所有子节点,孙子节点等. find_parents() 和 find_parent() 用来搜索当前节点的父辈节点,搜索方法与普通tag的搜索方法相同,搜索文档搜索文档包含的内容

### (4) find_next_siblings()  find_next_sibling()

### (5) find_previous_siblings()  find_previous_sibling()

### (6) find_all_next()  find_next()

### (7) find_all_previous() 和 find_previous()

## 四、CSS选择器

在CSS中，我们使用CSS选择器来定位节点。   

例如，div 节点的 id 为 container，那么就可以表示为 **#container**，其中 # 开头代表选择  id，其后紧跟id的名称。   
另外，如果我们想选择 class 为 wrapper 的节点，便可以使用 **.wrapper**，这里以点（.）开头代表选择 class ，其后紧跟 class 的名称。   
另外，还有一种选择方式，那就是根据标签名筛选，例如想选择二级标题，直接用h2即可。这是最常用的3种表示，分别是根据id、class、标签名筛选。

**CSS选择器还支持嵌套选择**，各个选择器之间加上**空格**分隔开便可以代表**嵌套**关系。    
如 **#container .wrapper p** 则代表先选择 id 为 container 的节点，然后选中其内部的 class 为 wrapper 的节点，然后再进一步选中其内部的 p 节点。

**如果不加空格，则代表并列关系**，如 **div#container .wrapper p.text** 代表先选择 id 为 container 的div节点，然后选中其内部的class为wrapper的节点，再进一步选中其内部的class为text的p节点。

在这里我们也可以利用类似的方法来筛选元素，用到的方法是 soup.select()，返回类型是 list

### (1) 通过标签名查找

In [88]:
soup.select('title') 

[<title>The Dormouse's story</title>]

In [89]:
soup.select('a')

[<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

In [90]:
soup.select('b')

[<b>The Dormouse's story</b>]

### (2) 通过类名查找

In [91]:
soup.select('.sister')

[<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

### (3) 通过 id 名查找

In [92]:
soup.select('#link1')

[<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>]

### (4) 组合查找     
组合查找即和写 class 文件时，标签名与类名、id名进行的组合原理是一样的，例如查找 p 标签中，id 等于 link1的内容，二者需要用空格分开

In [93]:
soup.select('p #link1')

[<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>]

In [94]:
# 直接子标签查找
soup.select("head > title")

[<title>The Dormouse's story</title>]

### (5) 属性查找    
查找时还可以加入属性元素，属性需要用中括号括起来，注意属性和标签属于同一节点，所以中间不能加空格，否则会无法匹配到。

In [95]:
soup.select('a[class="sister"]')

[<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

In [96]:
soup.select('a[href="http://example.com/elsie"]')

[<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>]

同样，属性仍然可以与上述查找方式组合，不在同一节点的空格隔开，同一节点的不加空格

In [97]:
soup.select('p a[href="http://example.com/elsie"]')

[<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>]

以上的 select 方法返回的结果都是列表形式，可以遍历形式输出，然后用 get_text() 方法来获取它的内容。

In [99]:
soup = BeautifulSoup(html)
print(type(soup.select('title')))
print(soup.select('title')[0].get_text())
 
for title in soup.select('title'):
    print(title.get_text())

<class 'list'>
The Dormouse's story
The Dormouse's story


# 实战项目

## 1.爬取链家租房的信息

In [3]:
import requests
from bs4 import BeautifulSoup

In [58]:
#获取url页面内容
def get_page(url,headers):
    r = requests.get(url, headers = headers)
    r.encoding = r.apparent_encoding
    soup = BeautifulSoup(r.text)
    return soup

#获取整页的每个租房信息的链接
def get_links(link_url,headers):
    soup = get_page(link_url,headers)
    links_a = soup.find_all('a' , attrs = {'class':'content__list--item--aside'})
    links = [('https://cd.lianjia.com'+ link.get('href')) for link in links_a]
    return links

In [49]:
#获取房屋的数据
def get_house_info(house_url):
    headers['Referer']=house_url
    soup = get_page(house_url,headers)
    try:
        name = soup.find('p', class_ = 'content__title').text
        price = soup.find('p', class_ = 'content__aside--title').text.strip()
        #text与.string的区别 https://zhuanlan.zhihu.com/p/30911642
        #个人建议基本用.text不容易出错

        #unit = soup.find('span', class_ = 'unit').text.strip()  #价格单位并去除首尾格式
        house_tag0 = soup.find('p', class_ = 'content__aside--tags').text.strip().split('\n')
        house_tag = ''
        for tag in house_tag0 :
            house_tag = house_tag + tag + ' '

        house_info = soup.find('p', class_ = 'content__article__table').text.strip().split('\n')
        way = house_info[0]
        layout = house_info[1]
        area = house_info[2]
        direction = house_info[3]

        #楼层信息在房屋信息栏里，需要切片访问
        floor = soup.find('div',  class_  =  'content__article__info').text.strip().split('\n')[9][3:]
    except:
        return {}
    info = {
        '标题':name,
        '价格': price,
        '标签':house_tag,        
        '租法':way,
        '户型':layout,
        '面积':area,        
        '朝向':direction, 
        '楼层':floor,
    }
    return info


In [55]:
#抓取的文件写入本地
def outToFile(dict):
    '''
    写入本地当前目录的cdlianjia.txt文件中
    '''
    if dict:
        with open('cdlianjia.txt','a+',encoding='utf-8') as f:
            f.write('标题:{}\t 价格:{}\t 标签:{}\t 租法:{}\t 户型:{}\t 面积:{}\t 朝向:{}\t 楼层:{}\n'.
                    format(dict['标题'],dict['价格'], dict['标签'], dict['租法'], dict['户型'], 
                           dict['面积'], dict['朝向'], dict['楼层']))
            


In [59]:
def start(deep):
    #构造headers模拟真实浏览器访问
    headers = {
        'Host': "cd.lianjia.com",
        'Accept': "application/json, text/javascript, */*; q=0.01",
        'Accept-Encoding': "gzip, deflate, br",
        'Accept-Language': "zh-CN,zh;q=0.9",
        'Referer': 'https://cd.lianjia.com/zufang/',
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36",
        'Connection': "keep-alive",
    }
    url_list = [('https://cd.lianjia.com/zufang/'+'pg'+str(k) +'/') for k in range(1,deep+1)]
    print('已获取链接，开始爬虫')
    for url in url_list:
        headers['Referer'] = url
        links = get_links(url, headers)  #得到每个房屋的链接        
        for link in links :
            house_info = get_house_info(link)  #得到单个租房信息字典
            outToFile(house_info)
        print('当前网页爬取完毕')
    print('所有页已爬取完毕')
        

In [60]:
start(4)

已获取链接，开始爬虫
当前网页爬取完毕
当前网页爬取完毕
当前网页爬取完毕
当前网页爬取完毕
所有页已爬取完毕


* **错误总结：**    
1. get('href')写成了get('herf')，于是截取不到链接    
2. 'NoneType' object has no attribute 'text'错误，是我们提取的时候有空的信息，要进行判断处理

## 2.爬取百度贴吧逆水寒吧的帖子信息

In [62]:
import requests
import time
from bs4 import BeautifulSoup

In [63]:
#获取网页

def get_html(url):
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        # 这里我们知道百度贴吧的编码是utf-8，所以手动设置的。爬去其他的页面时建议使用如下代码，但是该编码方式不一定准确：
        #r.encoding = r.apparent_encoding
        r.encoding = 'utf-8'
        return r.text
        #html=r.content
        #html_doc=str(html,'utf-8')         #html_doc=html.decode("utf-8","ignore")
        #return html_doc
    except:
        return " ERROR "

这里我们知道 r.text 返回的是Unicode型的数据。    
使用r.content返回的是bytes型的数据。   
也就是说，如果你想取文本，可以通过r.text。  
如果想取图片，文件，则可以通过r.content。   

<pre/>
    r = requests.get(url, timeout = 30)
    r.raise_for_status()
    html=r.content
    html_doc=str(html,'utf-8')         #html_doc=html.decode("utf-8","ignore")
    return html_doc
</pre>

In [64]:
#获取信息
def get_content(url):
    
    #初始化一个列表用来保存所有帖子信息
    posts = []
    #下载网页到本地
    html_doc = get_html(url)

    
    
    #a = requests.get(url)
    #html=a.content
    #html_doc=str(html,'utf-8') 
    #html_doc=html.decode("utf-8","ignore")
    #做一锅汤
    soup = BeautifulSoup(html_doc)

    #找到所有具有 'j_thread_list clearfix'属性的li标签，存入列表
    liTags = soup.find_all('li', attrs={'class':'j_thread_list'} )

    #寻找信息

    for li in liTags:
        #字典存储文章信息
        post = {}

        #使用try 防止爬虫找不到信息从而停止运行
        try :
            '''
            post['title'] = li.find('a', attrs = {'class':'threadlist_title pull_left j_th_tit '}).text.strip()
            post['link'] = "http://tieba.baidu.com/"+li.find('a', attrs={'class': 'threadlist_title pull_left j_th_tit '})['href']
            post['name'] = li.find('span', attrs={'class': 'tb_icon_author '}).text.strip()
            post['time'] = li.find('span', attrs = {'class':'threadlist_reply_date pull_right j_reply_data'}).text.strip()
            post['replyNum'] = li.find('span', sttrs = {'class':'threadlist_reply_date pull_right j_reply_data'}).text.strip()
            '''
            post['title'] = li.find('a', class_='j_th_tit').text
            post['link'] = 'http://tieba.baidu.com' + li.find('a', class_='j_th_tit').get('href')
            post['name'] = li.find('a', class_= 'j_user_card').text
            post['time'] = li.find('span', class_='is_show_create_time').text
            post['replyNum'] = li.find('span', class_='threadlist_rep_num').text
            posts.append(post)
        except:
            print('出问题了~')
    return posts

In [65]:
#抓取的文件写入本地
def outToFile(dict):
    '''
    写入本地当前目录的nshba.txt文件中
    '''
    with open('nshba.txt','a+',encoding='utf-8') as f:
        for post in dict:
            f.write('标题:{}\t 链接:{}\t 发帖人:{}\t 最后回复时间:{}\t 回复数量:{}\n'.format(
                post['title'], post['link'], post['name'], post['time'], post['replyNum']))
            
        print('当前网页爬取完毕')

In [66]:
def start(url_0, deep):
    
    url_list = [] #将要爬取的url存入列表

    for i in range(0,deep):
        url_list.append(url_0 +'&pn=' + str(50 * i))
    print('所有网页已经下载到本地了，开始爬虫吧')
    
    #循环写入信息
    for url in url_list:
        post = get_content(url)
        outToFile(post)
    print('所有信息都爬取并保存完毕')

In [67]:
url_0 = 'https://tieba.baidu.com/f?kw=%E9%80%86%E6%B0%B4%E5%AF%92ol&ie=utf-8'
deep = 3
start(url_0, deep)

所有网页已经下载到本地了，开始爬虫吧
当前网页爬取完毕
当前网页爬取完毕
当前网页爬取完毕
所有信息都爬取并保存完毕


## 3.爬取小说网站排行榜小说的链接

In [2]:
'''
小说网站: https://www.qu.la/
排行榜所在网页：https://www.qu.la/paihangbang/
爬取的信息：排行榜单，链接
'''
pass

In [3]:
import requests
from bs4 import BeautifulSoup
import time
import csv

In [4]:
#获取网页soup

def get_page(url):
    try:
        r = requests.get(url, timeout = 30)
        r.raise_for_status
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return('Wrong in get_page()')

In [5]:
#获取排行榜小说的名字和链接

def get_content(url):
    url_list = []
    html = get_page(url)
    soup = BeautifulSoup(html)
    
    category_list = soup.find_all('div', class_ = 'index_toplist')
    
    #使用w+方式，新建读写文件，覆盖原文件
    with open('novel_list.csv','w+') as f:
        pass
    
    for cate in category_list:
        #排行榜名
        name = cate.find('div', class_ = 'toptab').span.text  
        with open('novel_list.csv','a+') as f:
            f.write('\n小说种类：{}\n'.format(name))
        
        #通过style属性定位排行榜
        general_list = cate.find(style = 'display: block;')
        #小说名和链接包含在li标签里
        book_list = general_list.find_all('li')
        
        for book in book_list:
            link = 'http://www.qu.la' + book.a.get('href')
            title = book.a.get('title')
            
            #所有文章链接保存在列表里
            url_list.append(link)
            #使用a模式防止清空文件
            with open('novel_list.csv', 'a', newline = '') as f:
                writer = csv.writer(f)
                writer.writerow(['小说名：{:<}'.format(title),
                                '小说链接：{:<}'.format(link)])

        return(url_list)

In [6]:
# 获取单本小说所有章节链接，并创建小说文件

def get_txt_url(url):
    url_list = []
    html = get_page(url)
    soup = BeautifulSoup(html)
    #章节
    list_dd = soup.find_all('dd')
    txt_name = soup.find('div', id = 'info').h1.text
    with open('D:\download\小说\{}.txt'.format(txt_name),'a+') as f:
        f.write('小说标题：{}'.format(txt_name))
    for url in list_dd:
        url_list.append('http://www.qu.la'+url.a['href'])
    del(url_list[0:12])
    return url_list, txt_name

In [31]:
# 获取单章内容并保存本地

def get_one_txt(url, txt_name):
    html = get_page(url)
    soup = BeautifulSoup(html)
    
    try:
        title = soup.find('div', class_ = 'bookname').h1.text
        txt = title +'\n'+soup.find(
            'div', id = 'content').text.replace('chaptererror();',' ').replace('\xa0','').replace('\ufeff','')
        with open('D:\download\小说\{}.txt'.format(txt_name),'a+') as f:
            f.write(txt)
            print('当前小说：{}的章节：{}已下载完成'.format(txt_name,title))
    except:
        print('something wrong in get_one_txt()')

In [35]:
def start():
    url_0 = 'https://www.qu.la/paihangbang/'
    book_url_list = get_content(url_0)
    i,j=0,0
    for book_url in book_url_list:
        txt_url_list, txt_name = get_txt_url(book_url)
        for txt_url in txt_url_list:
            get_one_txt(txt_url, txt_name)
            j = j+1
            if(j>=15):
                j=0
                break
        i = i + 1
        if(i >=4):
            break

In [ ]:
start()

* **错误总结：**    
1. 使用writerow时，每行之间多了一个空格，这是因为在windows这种使用\r\n的系统里，不用newline=''的话，会自动在行尾多添加个\r，导致多出一个空行，即行尾为\r\r\n     
    * 解决办法：增加newline=''参数：with open('novel_list.csv','a'， newline = '') as f:    
2. 使用writerow时，如果直接传入字符串对象，写入会变为一个字符一列！！ 
    * 解决办法：传入的参数应该改为字符串列表！！   
3. get_content(url)函数的return参数没有加()括号，导致返回NULL    
4. get_one_txt()函数出现错误：UnicodeEncodeError: 'gbk' codec can't encode character '\xa0' in position 12: illegal multibyte sequence， 原因见：['\xa0'编码问题](https://blog.csdn.net/github_35160620/article/details/53353672)和[Python里编码解码问题](https://blog.csdn.net/jim7424994/article/details/22675759)    
    * 解决办法：用''来替换'\xa0'和'\ufeff'， 同时注意utf-8到unicode编码时有chaptererror();也需要替换

## 4.电影排行榜和图片批量下载

In [1]:
'''
网站：http://dianying.2345.com/top/
爬取的信息：电影名字，主演，简介，标题图
'''
pass

In [74]:
import requests
from bs4 import BeautifulSoup
import csv

In [48]:
# 获取网页
def get_page(url):
    r = requests.get(url)
    r.encoding = 'gbk'
    return r.text

In [ ]:
#图片下载的代码
def get_pic_from_url(url):
    #从url以二进制的格式下载图片数据
    pic_content = requests.get(url,stream=True).content
    open('filename','wb').write(pic_content)

In [99]:
def get_list(url):
    movies_list  = []
    r = get_page(url)
    soup = BeautifulSoup(r)
    movies_list = soup.find('ul', class_ = 'picList clearfix').find_all('li')
    movies = []
    
    #写入行名，使用w+直接清空重写
    #常用 / 写相对路径，用 \ 写绝对路径
    with open('D:\download\movies\movies.csv','w',newline = '') as f:
        writer = csv.writer(f)
        writer.writerow(['电影名','主演','简介','海报'])
        writer.writerow('')
    
    for movie in movies_list:
        name = movie.find('span', class_ = 'sTit').text.strip()

        #每位演员名字的分割？
        actors = ''
        #tag 的.content 属性可以将tag的子节点以列表的方式输出
        actors0 = movie.find('p', class_ = 'pActor').contents
        for actor in actors0:
            actors = actors+actor.string+' '
        actors = actors[4:].strip()

        if movie.find('p',class_ = 'pTxt pIntroHide'):
            brief = movie.find('p',class_ = 'pTxt pIntroHide').text[3:-7].strip()
        else:
            brief = movie.find('p',class_ = 'pTxt pIntroShow').text[3:].strip()
        
        
        # 获取图片
        
        #这里提取src可以用.get('src')，也可以直接movie.find('img')['src']
        img_url = 'http:'+movie.find('img').get('src')
        with open('D:\download\movies\ '+name+'.png','wb+') as f:
            f.write(requests.get(img_url, stream = True).content)
            
        #将电影信息写入文件
        with open('D:\download\movies\movies.csv','a+',newline = '') as f:
            writer = csv.writer(f)
            writer.writerow([name,actors,brief,img_url])
                             
        movies.append({
            '电影名':name,
            '主演':actors,
            '简介':brief
        })  
        
    return movies

In [94]:
def start():
    url = 'http://dianying.2345.com/top/'
    get_list(url)

In [100]:
start()

* **错误总结**    
* 编码问题：使用r.apparent_encoding测试是GB2312，对某些生僻字的显示出现了乱码，使用GBK可以解决    
* 电影简介有的自带完整版，有的需要点击展开全部之后才显示完整版，如何直接显示所有的完整版简介？    
    * if条件判断一下    
* 电影主演如果直接获取tag的text，名字之间没有了分割，如何将每位演员名字分隔开？
    * 注意.contents的用法和字符串拼接    
* with open('D:\download\movies\ '+name+'.png','wb+') as f: 此处第一个路径字符串的最后一个\之后必须跟一个字符 再连接name 才不会报错，不知是什么原因？

## 5.爬取音悦台榜单

In [1]:
'''
网站：http://vchart.yinyuetai.com/vchart/trends?
获取信息：音悦台的各个地区前20榜单，包含歌曲名，歌手，发布时间，评分，趋势
'''
pass

In [2]:
import requests
import csv
from bs4 import BeautifulSoup

In [85]:
def get_page(url):
    try:
        r = requests.get(url)
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return('Wrong in get_page('+url+')!')

In [94]:
def get_list(url0):
    areas = ['ALL', 'ML', 'HT', 'US', 'KR', 'JP']
    areas_names = ['总榜', '内地篇', '港台篇', '欧美篇', '韩国篇', '日本篇']
    #将地区代号和地区名打包成字典
    area_dict = dict(zip(areas,areas_names))
    with open('D:\download\music\yinyuetai.csv','w+') as f:
        writer = csv.writer(f)
        writer.writerow(['地区', '歌曲名', '歌手', '发布时间','评分','趋势'])
        
    for area in areas:
        area_name = area_dict[area]
        with open('D:\download\music\yinyuetai.csv','a+') as f:
            writer = csv.writer(f)
            writer.writerow([area_name,])
        
        #得到不同地区的网页
        url = url0 + 'area='+area
        r = get_page(url)
        soup = BeautifulSoup(r)
        music_list = soup.find_all('li', class_ = 'vitem J_li_toggle_date')
        for music in music_list:
            music_name = music.find('div', class_ = 'info').h3.text.strip()
            
            #歌手名字分割
            music_singer_list = music.find('p', class_ ='cc').find_all('a')
            music_singer = ''
            for singer in music_singer_list:
                music_singer = music_singer+singer.string.strip()+' & '
            music_singer = music_singer[:-3]
            
            music_time = music.find('p', class_ = 'c9').text[5:]
            music_score = music.find('h3').text
            music_tendency = music.find('span').text
            
            with open('D:\download\music\yinyuetai.csv','a+',newline = '') as f:
                writer = csv.writer(f)
                writer.writerow(['', music_name, music_singer, music_time, 
                                 music_score,music_tendency])

In [87]:
def start():
    url = 'http://vchart.yinyuetai.com/vchart/trends?'
    get_list(url)

In [95]:
start()

* **错误总结：**
* 歌手名字分割
* 注意class里复制的时候，末尾的空格问题